In [1]:
import pandas as pd
import numpy as np

In [8]:
raw_df = pd.read_csv("../concept_labeling_outputs/merged_dataset_with_concepts.csv")
window_df = pd.read_csv("../concept_labeling_outputs/merged_window_labels.csv")

In [9]:
print(raw_df.shape)
print(window_df.shape)

(8802, 10)
(150, 8)


In [11]:
raw_df["magnitude"] = np.sqrt(raw_df["x-axis"]**2 + raw_df["y-axis"]**2 + raw_df["z-axis"]**2)

raw_df["motion_intensity"] = (raw_df["magnitude"] - raw_df["magnitude"].min()) / (raw_df["magnitude"].max() - raw_df["magnitude"].min())

x_norm = np.linalg.norm(raw_df["x-axis"])
y_norm = np.linalg.norm(raw_df["y-axis"])
z_norm = np.linalg.norm(raw_df["z-axis"])

raw_df["vertical_dominance"] = abs(raw_df["z-axis"]) / (abs(raw_df["x-axis"]) + abs(raw_df["y-axis"]) + abs(raw_df["z-axis"]) + 1e-9)

raw_df["static_posture"] = (raw_df["motion_intensity"] < 0.1).astype(float)

def compute_window_features(df, start, end):
    segment = df[(df["time_s"] >= start) & (df["time_s"] <= end)]
    if len(segment) == 0:
        return pd.Series({"motion_intensity": np.nan, "vertical_dominance": np.nan, "static_posture": np.nan, "magnitude": np.nan})
    mag = np.sqrt(segment["x-axis"]**2 + segment["y-axis"]**2 + segment["z-axis"]**2)
    motion_intensity = (mag.mean() - raw_df["magnitude"].min()) / (raw_df["magnitude"].max() - raw_df["magnitude"].min())
    vert_dom = np.linalg.norm(segment["z-axis"]) / (
        np.linalg.norm(segment["x-axis"]) + np.linalg.norm(segment["y-axis"]) + np.linalg.norm(segment["z-axis"]) + 1e-9
    )
    static_posture = 1.0 if motion_intensity < 0.1 else 0.0
    magnitude_mean = mag.mean()
    return pd.Series({
        "motion_intensity": motion_intensity,
        "vertical_dominance": vert_dom,
        "static_posture": static_posture,
        "magnitude": magnitude_mean
    })

window_features = window_df.apply(lambda row: compute_window_features(raw_df, row["start_time"], row["end_time"]), axis=1)
window_df = pd.concat([window_df, window_features], axis=1)

raw_df.drop(columns=["magnitude"], inplace=True)
window_df.drop(columns=["magnitude"], inplace=True)

raw_df.to_csv("raw_with_features.csv", index=False)
window_df.to_csv("window_with_features.csv", index=False)

In [12]:
print(raw_df[["motion_intensity", "vertical_dominance", "static_posture"]].describe())
print(window_df[["motion_intensity", "vertical_dominance", "static_posture"]].describe())

print(raw_df.shape)
print(window_df.shape)

       motion_intensity  vertical_dominance  static_posture
count       8802.000000         8802.000000     8802.000000
mean           0.331265            0.243082        0.032947
std            0.146701            0.213129        0.178508
min            0.000000            0.000000        0.000000
25%            0.270740            0.084025        0.000000
50%            0.295913            0.177345        0.000000
75%            0.387405            0.358235        0.000000
max            1.000000            0.916763        1.000000
       motion_intensity  motion_intensity  vertical_dominance  \
count        150.000000        150.000000          150.000000   
mean           0.331157          0.331157            0.248287   
std            0.041124          0.041124            0.080944   
min            0.277237          0.277237            0.041401   
25%            0.304296          0.304296            0.201228   
50%            0.316702          0.316702            0.236932   
75%  

In [13]:
print(raw_df.head())
print(window_df.head())

   user activity        timestamp  x-axis  y-axis  z-axis      time_s  \
0     3  Walking  101790131507000    1.76    0.46   -1.61  957.780029   
1     3  Walking  101790211616000    0.57    1.50   -2.30  957.860138   
2     3  Walking  101790251594000    2.56    5.48    5.52  957.900116   
3     3  Walking  101790291480000   11.73   14.90   17.08  957.940002   
4     3  Walking  101790331550000    9.43    5.01    3.60  957.980072   

   periodicity  temporal_stability  coordination  motion_intensity  \
0          1.0                 0.5           0.5          0.063145   
1          1.0                 0.5           0.5          0.075010   
2          1.0                 0.5           0.5          0.245287   
3          1.0                 0.5           0.5          0.793429   
4          1.0                 0.5           0.5          0.342695   

   vertical_dominance  static_posture  
0            0.420366             1.0  
1            0.526316             1.0  
2            0.40708